### Apply model on vdo having no class


In [1]:
import keras
from keras.models import Sequential
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, InputLayer, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import pickle


import cv2
import math
import os
from glob import glob
from scipy import stats as s

from moviepy.editor import *   ###VideoFileClip


## pafy, youtube-dl and moviepy packages to be installed
##pip install pafy youtube-dl moviepy

## Install open cv - make sure numpy already installed
#pip install opencv-python
#import pafy

 




C:\Users\jaghos\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\jaghos\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
C:\Users\jaghos\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\.libs\libopenblas.IPBC74C7KURV7CB2PKT5Z5FNR3SIBV4J.gfortran-win_amd64.dll
  stacklevel=1)
Using TensorFlow backend.


In [2]:


Drive = "C:"
## Define root folder
RootFolder = Drive+"/1-GG/CAP4/EventDetection/1-ExampleSetting/OutputFiles"

trainFrames  = pd.read_csv(RootFolder+"/trainFrames.csv")

trainFrames.head()


,FrameFilename,FullPathName,class
0,MoreBasketBallVdo.mp4,C:/1-GG/CAP4/EventDetection/1-ExampleSetting\V...,Basketball
1,MoreBasketBallVdo.mp4,C:/1-GG/CAP4/EventDetection/1-ExampleSetting\V...,Basketball
2,MoreBasketBallVdo.mp4,C:/1-GG/CAP4/EventDetection/1-ExampleSetting\V...,Basketball
3,MoreBasketBallVdo.mp4,C:/1-GG/CAP4/EventDetection/1-ExampleSetting\V...,Basketball
4,MoreBasketBallVdo.mp4,C:/1-GG/CAP4/EventDetection/1-ExampleSetting\V...,Basketball


In [3]:
# Get the unique values of 'class' column - How many distinct classes
model_class = trainFrames['class'].unique().tolist()
print(model_class)
model_output_size = len(model_class)
print(model_output_size)

['Basketball', 'SoccerPenalty']
2


## Pre Processing 

In [4]:
## VGG-16 pre-trained model will be used
# creating the base model of pre-trained VGG16 model
base_model = VGG16(weights='imagenet', include_top=False)

In [5]:
# Getting the YouTube Video's path you just downloaded
output_directory = 'C:/1-GG/CAP4/EventDetection/1-ExampleSetting'   # where is my VDO file ?

##video_title = "combined_3video"
video_title = "combined_video-2vdo"

input_video_file_path = f'{output_directory}/{video_title}.mp4'
input_video_file_path


'C:/1-GG/CAP4/EventDetection/1-ExampleSetting/combined_video-2vdo.mp4'

In [6]:
# Setting sthe Window Size which will be used by the Rolling Average Proces
window_size = 1

# Constructing The Output YouTube Video Path
#output_video_file_path = f'{output_directory}/{video_title} -Output-WSize {window_size}.mp4'

#print(output_video_file_path)

In [7]:
    %%time
    image_height  = 224
    image_width   = 224
    
    # Initialize a Deque Object with a fixed size which will be used to implement moving/rolling average functionality.
    #predicted_labels_probabilities_deque = deque(maxlen = window_size)

    # Reading the Video File using the VideoCapture Object
    video_reader = cv2.VideoCapture(input_video_file_path)

    # Getting the width and height of the video 
    original_video_width = int(video_reader.get(cv2.CAP_PROP_FRAME_WIDTH))
    original_video_height = int(video_reader.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Writing the Overlayed Video Files Using the VideoWriter Object
    #video_writer = cv2.VideoWriter(output_video_file_path, cv2.VideoWriter_fourcc('M', 'P', '4', 'V'), 24, (original_video_width, original_video_height))


    prediction_images = []
    while True: 

        # Reading The Frame
        status, frame = video_reader.read() 

        if not status:
            break

        #print("we are here")
        # Resize the Frame to fixed Dimensions
        resized_frame = cv2.resize(frame, (image_height, image_width))
        resized_frame = image.img_to_array(resized_frame)
        # Normalize the resized frame by dividing it with 255 so that each pixel value then lies between 0 and 1
        normalized_frame = resized_frame / 255

        
        
        # converting the list to numpy array
        # appending the image to the image list
        prediction_images.append(normalized_frame)
    # converting the list to numpy array
    prediction_images = np.array(prediction_images)
        
    # shape of the array
    prediction_images.shape
    # extracting features for validation frames
    prediction_images = base_model.predict(prediction_images)
    prediction_images.shape
 

Wall time: 59 s


(2673, 7, 7, 512)

In [8]:
# converting features in one dimensional array
prediction_images = prediction_images.reshape(prediction_images.shape[0], 7*7*512)


In [9]:
max = prediction_images.max()
prediction_images = prediction_images/max
prediction_images.shape

(2673, 25088)

## End of Pre Processing 

## Event Prediction 

### Load  model from disk

In [10]:
RootFolder

'C:/1-GG/CAP4/EventDetection/1-ExampleSetting/OutputFiles'

In [11]:
model_name = 'video combined_video-2vdo model_jj.h5'

from tensorflow import keras
model = keras.models.load_model(RootFolder+'/'+model_name)

In [12]:
predict = []        
# predict_classes() function model in order to predict the class values for each instance in the array.
prediction = model.predict_classes(prediction_images)
# prediction is an arrary containing frame by frame class prediction - here either Basketball', 'SoccerPenalty'

print(prediction)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
[1 1 1 ... 1 1 1]


#### Pickle is the standard way of serializing objects in Python.

#### Use the pickle operation to serialize  machine learning algorithms and save the serialized format to a file.

#### Later we can load this file to deserialize your model and use it to make new predictions.

#### Serialization is the process of converting an object into a stream of bytes to store the object or transmit it to memory, or a file.

### Save prediction array using pickle dump

In [13]:
import pickle

with open('combinedPrediction_j', 'wb') as fp:
    pickle.dump(prediction, fp)

In [14]:
#To read it back:
with open ('combinedPrediction_j', 'rb') as fp:
    prediction = pickle.load(fp)

In [15]:
prediction

array([1, 1, 1, ..., 1, 1, 1], dtype=int64)

### Function To Predict on Live Videos 

In [16]:
output_directory = 'C:/1-GG/CAP4/EventDetection/1-ExampleSetting'



video_title = "combined_video-2vdo"

input_video_file_path = f'{output_directory}/{video_title}.mp4'
input_video_file_path


# Constructing The Output YouTube Video Path
output_video_file_path = f'{output_directory}/{video_title} -2vdoOutput.mp4'

print(output_video_file_path)

C:/1-GG/CAP4/EventDetection/1-ExampleSetting/combined_video-2vdo -2vdoOutput.mp4


In [17]:
model_class

['Basketball', 'SoccerPenalty']

In [18]:
prediction[2000]

0

In [19]:
x = len(prediction)
print(x)
# Accessing The Class Name using predicted label.
predicted_class_name = model_class[prediction[2]]
predicted_class_name

2673


'SoccerPenalty'

In [20]:
input_video_file_path

'C:/1-GG/CAP4/EventDetection/1-ExampleSetting/combined_video-2vdo.mp4'

In [21]:
output_video_file_path

'C:/1-GG/CAP4/EventDetection/1-ExampleSetting/combined_video-2vdo -2vdoOutput.mp4'

In [22]:
def predict_on_live_video(video_file_path, output_file_path):

    # Initialize a Deque Object with a fixed size which will be used to implement moving/rolling average functionality.
    #predicted_labels_probabilities_deque = deque(maxlen = window_size)
    
    frame_no = -1

    # Reading the Video File using the VideoCapture Object
    video_reader = cv2.VideoCapture(video_file_path)

    # Getting the width and height of the video 
    original_video_width = int(video_reader.get(cv2.CAP_PROP_FRAME_WIDTH))
    original_video_height = int(video_reader.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Writing the Overlayed Video Files Using the VideoWriter Object
    video_writer = cv2.VideoWriter(output_file_path, cv2.VideoWriter_fourcc('M', 'P', '4', 'V'), 24, (original_video_width, original_video_height))

    while True: 

        # Reading The Frame
        status, frame = video_reader.read() 

        if not status:
            break

        frame_no = frame_no+1
        
        # some frames with black - skip those
        ## images in openCV (or in your case frames) are represented as a numpy array, 
        ## they can be averaged for low values (which represent black frames).
        
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  ## Convert to grey image (black and white)
        if np.average(gray) < 20:  ## if it dark screen , skip 
        # skips an iteration, so the frame isn't saved
          continue
        

           
        # Accessing The Class Name using prediction list.
        predicted_class_name = model_class[prediction[frame_no]]
        #print(predicted_class_name)
        
        # Overlaying Class Name Text Ontop of the Frame
        cv2.putText(frame, predicted_class_name, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        #cv2.putText(frame, predicted_class_name, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        #cv2.putText(frame, avg_prob, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
        
        #cv2.putText(frame, 
                    #avg_prob, 
                    #(10, 100),   # bottomLeftCornerOfText
                    #cv2.FONT_HERSHEY_SIMPLEX, 
                    #1, 
                    #(0, 0, 255), 
                    #2)
                       
        # Writing The Frame
        video_writer.write(frame)
    
    # Closing the VideoCapture and VideoWriter objects and releasing all resources held by them. 
    video_reader.release()
    video_writer.release()

In [23]:
# Calling the predict_on_live_video method to start the Prediction.
predict_on_live_video(input_video_file_path, output_video_file_path)
print("output video file created")

# Play Video File in the Notebook
#VideoFileClip(input_video_file_path).ipython_display(width = 700)

output video file created


### End